# MECP2 wikipathways:WP4312 DataNode Type Counts

In [11]:
from pip._internal import main as pip

try:
    import csv
except ImportError:
    pip(['install', 'csv'])
    import csv
try:
    from SPARQLWrapper import SPARQLWrapper, JSON
except  ImportError:
    import sys
    !{sys.executable} -m pip install SPARQLWrapper
    from SPARQLWrapper import SPARQLWrapper, JSON
    
import pandas
    
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
DataNodeCount = pandas.DataFrame(columns=['DataNode Type', 'DataNode Count'])

pathwayQuery = '''
      SELECT Distinct (substr(str(?o),41) as ?DNType) (COUNT(?o) as ?DNCount)
      WHERE {
        ?entity a ?o . 
        ?entity a wp:DataNode . 
        ?entity dcterms:isPartOf <http://identifiers.org/wikipathways/WP4312_r104287> .
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
         
      } 
      ORDER BY DESC (?DNCount)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	DataNodeCount = DataNodeCount.append({
		'DataNode Type': result["DNType"]["value"],
        'DataNode Count': result["DNCount"]["value"],
	 }, ignore_index=True)
    
DNCountInteger = DataNodeCount.loc[:, "DataNode Count"].apply(pandas.to_numeric, errors = 'ignore').sum()

In [12]:
DataNodeCount

,DataNode Type,DataNode Count
0,DataNode,152
1,GeneProduct,148
2,Protein,18
3,Rna,9
4,Complex,4


# MECP2 wikipathways:WP4312 GPMLRDF datanodes WITHOUT a WPRDF datanode equivalent

In [24]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
GpmlWoWP = pandas.DataFrame(columns=['DataNode Type', 'DataNode Count'])

pathwayQuery = '''
      SELECT (STR(?typeIRI) AS ?type) (COUNT(?datanode) AS ?count) WHERE {
          ?datanode a gpml:DataNode ; 
            gpml:type ?typeIRI .
          ?datanode dcterms:isPartOf ?pathway ;
            dcterms:isPartOf <http://rdf.wikipathways.org/Pathway/WP4312_r104287> .
          ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
          MINUS { ?wpDatanode wp:isAbout ?datanode } .
       } GROUP BY ?typeIRI
         ORDER BY DESC(?count)

    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	GpmlWoWP = GpmlWoWP.append({
		'DataNode Type': result["type"]["value"],
        'DataNode Count': result["count"]["value"],
	 }, ignore_index=True)
    
GpmlWoWPInteger = GpmlWoWP.loc[:, "DataNode Count"].apply(pandas.to_numeric, errors = 'ignore').sum()

In [25]:
GpmlWoWP

,DataNode Type,DataNode Count
0,GeneProduct,1


# MECP2 wikipathways:WP4312 Count of Interaction Types

In [22]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntTypesCount = pandas.DataFrame(columns=['Interaction Type', 'Interaction Count'])

pathwayQuery = '''
      SELECT (substr(str(?o),41) as ?IntType) (COUNT(?o) as ?IntCount)
      WHERE {
        ?entity a ?o . 
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf <http://identifiers.org/wikipathways/WP4312_r104287> .
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
      } ORDER BY DESC(?IntCount)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	IntTypesCount = IntTypesCount.append({
		'Interaction Type': result["IntType"]["value"],
        'Interaction Count': result["IntCount"]["value"],
	 }, ignore_index=True)
    
IntCountInteger = IntTypesCount.loc[:, "Interaction Count"].apply(pandas.to_numeric, errors = 'ignore').sum()

In [23]:
IntTypesCount

,Interaction Type,Interaction Count
0,Interaction,45
1,DirectedInteraction,37
2,Binding,20
3,Stimulation,12
4,ComplexBinding,4
5,Inhibition,3


# MECP2 wikipathways:WP4312 GPML RDF Interactions WITHOUT a WP RDF equivalent

In [18]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
noWPINT = pandas.DataFrame(columns=['Interactions Counted'])

pathwayQuery = '''
    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT (COUNT(?Interaction) AS ?count) 
    WHERE {

          ?Interaction a gpml:Interaction .
          ?Interaction dcterms:isPartOf ?pathway .
          ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
          MINUS {?wpInteraction wp:isAbout ?Interaction}

        } 
          ORDER BY DESC(?count)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	noWPINT = noWPINT.append({
        'Interactions Counted': result["count"]["value"],
	 }, ignore_index=True)

In [19]:
noWPINT

,Interactions Counted
0,12796
